In [58]:
import pyspark

In [59]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# total data

In [105]:
days = sc.emptyRDD()
for i in range(1,31):
    day_num = str(i) if len(str(i))==2 else '0'+str(i)
    day_map = sc.textFile('file:///data/mobile/raw_data/day%s.txt' % day_num)
    days = days.union(day_map)
days.take(5)

['3B0AAFBD3213D6DB0CCB17EDEAE80C38,2859,sys,0,20:43:27,1,20:43:50,86423,0,0',
 '3B0AAFBD3213D6DB0CCB17EDEAE80C38,2859,sys,0,20:43:27,1,20:43:50,86423,0,0',
 'BB1657CFD3B4B7208674BEB7D26014B2,4803,usr,1,13:40:45,1,13:44:41,235,3304,5851',
 'BB1657CFD3B4B7208674BEB7D26014B2,4803,usr,1,13:47:10,1,13:47:22,12,398,268',
 'BB1657CFD3B4B7208674BEB7D26014B2,4803,usr,1,13:47:23,1,13:50:10,167,63044,34622']

In [ ]:
with open('/data/mobile/processed_data/all_days.txt','w') as f:
    for line in days.collect():
        f.write(line+'\n')

In [ ]:
1

In [44]:
days = sc.emptyRDD()
for i in range(1,31):
    day_num = str(i) if len(str(i))==2 else '0'+str(i)
    day_map = sc.textFile('file:///data/mobile/raw_data/day%s.txt' % day_num)\
    .map(lambda x:[item for i,item in enumerate(x.split(',')) if i in [0,1,7]])
    days = days.union(day_map)
days.take(5)

[['3B0AAFBD3213D6DB0CCB17EDEAE80C38', '2859', '86423'],
 ['3B0AAFBD3213D6DB0CCB17EDEAE80C38', '2859', '86423'],
 ['BB1657CFD3B4B7208674BEB7D26014B2', '4803', '235'],
 ['BB1657CFD3B4B7208674BEB7D26014B2', '4803', '12'],
 ['BB1657CFD3B4B7208674BEB7D26014B2', '4803', '167']]

In [45]:
connect_id_new = days.map(lambda x:(x[0]+'-'+x[1],int(x[2])))

In [46]:
connect_id_new.first()

('3B0AAFBD3213D6DB0CCB17EDEAE80C38-2859', 86423)

In [47]:
sum_time = connect_id_new.reduceByKey(lambda x,y:x+y)

In [48]:
sum_time.take(5)

[('DBBA78B45305FA654ECE4351A5389AB6-9098', 36446),
 ('0C137DDC6985349303B5CF58FF8B3B56-3427', 2747),
 ('9370DBB1A0CDC073480261B944C2EB20-20011', 20836),
 ('30B50DCCE309FCC62D900957F1DF5DE2-3309', 71470),
 ('A4B5FE476DE0CA626F4057ED08CBC8EF-20720', 4074)]

In [49]:
out_sum = sum_time.map(lambda x:','.join(x[0].split('-') + [str(x[1])]))

In [50]:
out_sum.saveAsTextFile('file:///data/mobile/processed_data/total_usage_time.txt')

In [53]:
count_freq = days.map(lambda x:(x[0]+'-'+x[1],1)).reduceByKey(lambda x,y:x+y)

In [54]:
out_freq = count_freq.map(lambda x:','.join(x[0].split('-') + [str(x[1])]))

In [55]:
out_freq.saveAsTextFile('file:///data/mobile/processed_data/total_usage_freq.txt')

In [56]:
with open('/data/mobile/processed_data/total_time.txt','w') as f:
    for line in out_sum.collect():
        f.write(line+'\n')

In [57]:
with open('/data/mobile/processed_data/total_freq.txt','w') as f:
    for line in out_freq.collect():
        f.write(line+'\n')

In [42]:
y.union(day).first()

'3B0AAFBD3213D6DB0CCB17EDEAE80C38,2859,sys,0,20:43:27,1,20:43:50,86423,0,0'

In [51]:
y = sc.textFile('file:///data/mobile/processed_data/total_usage_time.txt')

In [52]:
y.first()

'1537C1E68F14EDCF46221ADE8B4D4CE8,14663,135'

In [60]:
y.first()

'1537C1E68F14EDCF46221ADE8B4D4CE8,14663,135'

In [62]:
sum_time.first()

('DBBA78B45305FA654ECE4351A5389AB6-9098', 36446)

In [63]:
str16 = y.first().split(',')[0]
int(str16,16)

28203297010515189848969325948517043432

In [83]:
from pyspark.mllib.recommendation import Rating, ALS
ratings = sum_time.map(lambda x: Rating(int(x[0].split('-')[0],16)%2147483647,\
int(x[0].split('-')[1]) if x[0].split('-')[1] not in ['河南省郑州市二七区康复后街','河南省郑州市二七区大学北路25号'] else 0,\
                        float(x[1])/30/60/60 if float(x[1])/30/60/60 <=4 else 4.0))
ratings.collect()

[Rating(user=1824575646, product=9098, rating=0.33746296296296296),
 Rating(user=229088056, product=3427, rating=0.025435185185185182),
 Rating(user=1938132469, product=20011, rating=0.19292592592592592),
 Rating(user=1590678036, product=3309, rating=0.6617592592592593),
 Rating(user=1149211800, product=20720, rating=0.037722222222222226),
 Rating(user=1373414206, product=5657, rating=0.027472222222222224),
 Rating(user=1010783088, product=776, rating=0.004953703703703704),
 Rating(user=1276519403, product=10686, rating=0.18705555555555556),
 Rating(user=1817388541, product=20124, rating=0.01923148148148148),
 Rating(user=662042195, product=11088, rating=0.05162962962962963),
 Rating(user=2072729185, product=4803, rating=0.033046296296296296),
 Rating(user=1381789621, product=19799, rating=0.005194444444444444),
 Rating(user=122769352, product=13667, rating=0.05542592592592592),
 Rating(user=1640169778, product=4761, rating=0.005435185185185185),
 Rating(user=781912500, product=21140, 

In [84]:
model = ALS.train(ratings, 50, 10, 0.01)
userFeatures = model.userFeatures()
userFeatures.take(2)

[(8799840,
  array('d', [-0.0058010658249258995, -0.0023295897990465164, -0.0003298154624644667, -0.011134488508105278, 0.016496729105710983, 0.004202214069664478, -0.008479841984808445, -0.0004751080123241991, 0.007150671910494566, -0.00531047722324729, 0.012678666040301323, 0.00976538099348545, -0.008469296619296074, 0.0020643805619329214, 0.000815348990727216, -0.011953813955187798, 0.00012179344048490748, 0.008537647314369678, -0.0035914205946028233, -0.00438966229557991, 0.011838776059448719, 0.006778324954211712, 0.012341388501226902, 0.0004377104341983795, -0.0004198764218017459, -0.011654911562800407, -0.011748441495001316, 0.007194405421614647, 0.00502823106944561, 0.00253756414167583, -0.018299488350749016, 0.010088752023875713, 0.008825259283185005, 0.0037041986361145973, 0.0028601016383618116, 0.002971786307170987, 0.0019916242454200983, -0.0011705518700182438, 0.0006630285060964525, -0.020619791001081467, 7.518988422816619e-05, -0.005518254823982716, -0.008293166756629944,

In [85]:
print(model.userFeatures().count())
print(model.productFeatures().count())

48178
42676


In [86]:
print(len(userFeatures.first()[1]))
predictRating = model.predict(1824575646,9098)
print(predictRating)

50
0.3158283406607182


In [87]:
topKRecs = model.recommendProducts(1824575646,30)
print('给用户userId推荐其喜欢的item：')
for rec in topKRecs:
    print(rec)

给用户userId推荐其喜欢的item：
Rating(user=1824575646, product=16065, rating=2.0521232314607576)
Rating(user=1824575646, product=10062, rating=1.5718997814389193)
Rating(user=1824575646, product=19011, rating=1.5167092734937024)
Rating(user=1824575646, product=3309, rating=1.4786870356642021)
Rating(user=1824575646, product=27463, rating=1.4494365000575626)
Rating(user=1824575646, product=8471, rating=1.4250994393237097)
Rating(user=1824575646, product=13614, rating=1.358917440722543)
Rating(user=1824575646, product=15304, rating=1.3351248601503276)
Rating(user=1824575646, product=19031, rating=1.3167293627266377)
Rating(user=1824575646, product=19350, rating=1.2824455259594452)
Rating(user=1824575646, product=16258, rating=1.275055423397284)
Rating(user=1824575646, product=602, rating=1.2703005459650685)
Rating(user=1824575646, product=16719, rating=1.2690815126914419)
Rating(user=1824575646, product=10438, rating=1.2552557188271327)
Rating(user=1824575646, product=1105, rating=1.22307127042179

In [89]:
appsForUser = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(1824575646)
print('用户使用appid=%d的日均小时数'%len(appsForUser[0]))
print('源数据中用户(userId=789)使用的app(item)：')
for i in sorted(appsForUser[0],key=lambda x : x.rating,reverse=True):
    print('appid={0} usage_average_hours={1}'.format(i.product,i.rating))

用户使用appid=46的日均小时数
源数据中用户(userId=789)使用的app(item)：
appid=3309 usage_average_hours=1.6116388888888886
appid=11332 usage_average_hours=0.9463888888888888
appid=7280 usage_average_hours=0.9155185185185186
appid=9098 usage_average_hours=0.33746296296296296
appid=4803 usage_average_hours=0.16460185185185186
appid=11088 usage_average_hours=0.04331481481481481
appid=10686 usage_average_hours=0.028685185185185185
appid=13043 usage_average_hours=0.025083333333333332
appid=13667 usage_average_hours=0.024648148148148148
appid=22760 usage_average_hours=0.02400925925925926
appid=2292 usage_average_hours=0.02231481481481481
appid=6192 usage_average_hours=0.02183333333333333
appid=62 usage_average_hours=0.017129629629629627
appid=17355 usage_average_hours=0.015944444444444445
appid=17030 usage_average_hours=0.00837962962962963
appid=7540 usage_average_hours=0.006740740740740741
appid=2859 usage_average_hours=0.006212962962962963
appid=3427 usage_average_hours=0.005759259259259259
appid=6764 usage_ave

In [ ]:
sc

In [90]:
1

1

In [91]:
import pandas as pd
cluster_res = pd.read_csv('/data/mobile/processed_data/cluster_label.csv')
cluster_res

,uid,cluster_label
0,0000348F61881026FE0036840F25309F,0
1,00046AA87AAF8C9971F99597ED7D90DF,1
2,000BAC3E23007010346E79B1ED5D25E2,0
3,000C1F225DAAEDE0DF4FCAED6DE7EF3C,0
4,0015AE689CD28553CB5FD29E394C7D11,2
...,...,...
22239,FFF435B912C1104B29FA0B02D76122BD,0
22240,FFF62C1EEF706DF3174EF676391E15C1,2
22241,FFF82C898EE523DA1F4137097FECCB20,0
22242,FFFA25131B53BDF8B855AB4D473E29C2,0


In [92]:
uid2label = dict(zip(cluster_res['uid'].apply(lambda x:int(x,16)%2147483647),cluster_res['cluster_label']))
uid2label

{289899198: 0,
 1012024248: 1,
 1654209399: 0,
 596915629: 0,
 1141091041: 2,
 1963694919: 1,
 1355910941: 0,
 175616510: 2,
 2097783842: 2,
 914919552: 1,
 1521888537: 1,
 223016605: 0,
 878071896: 0,
 238904266: 0,
 668061245: 0,
 1394804627: 2,
 222742633: 2,
 446170493: 2,
 1597521729: 0,
 493372764: 0,
 1345833715: 2,
 2008053886: 2,
 1941520724: 1,
 480598273: 1,
 1025730182: 1,
 1826173502: 1,
 902684241: 2,
 1484787298: 0,
 934292810: 2,
 1403475871: 2,
 388912488: 2,
 846173289: 2,
 1134164587: 0,
 1192818900: 2,
 1432046541: 1,
 425711243: 0,
 785483296: 2,
 1244925688: 0,
 1262199292: 2,
 880145347: 0,
 705829005: 1,
 1139198886: 0,
 1235668830: 0,
 1694079671: 1,
 1729655116: 0,
 673679547: 1,
 1610729551: 0,
 76570905: 0,
 631191554: 1,
 1348582702: 0,
 524255604: 0,
 1710205464: 2,
 2088568369: 1,
 602109978: 2,
 490484483: 1,
 483474579: 0,
 1049710268: 2,
 1575898231: 2,
 1152062978: 1,
 589045396: 0,
 251778990: 0,
 2137083979: 2,
 1421122020: 1,
 1574953416: 2,
 13775

In [93]:
uid2label[1824575646]

2

In [98]:
import pickle
with open('/data/mobile/processed_data/appid2class.pk','rb') as f:
    appid2class = pickle.load(f)

In [99]:
appid2class

{247: 'a',
 475: 'a',
 749: 'a',
 1074: 'a',
 1077: 'a',
 1093: 'a',
 1460: 'a',
 1670: 'a',
 2704: 'a',
 2807: 'a',
 2853: 'a',
 2977: 'a',
 3662: 'a',
 4581: 'a',
 4694: 'a',
 4700: 'a',
 4803: 'a',
 4804: 'a',
 4887: 'a',
 4985: 'a',
 5173: 'a',
 5276: 'a',
 5613: 'a',
 5616: 'a',
 5726: 'a',
 5904: 'a',
 5964: 'a',
 5966: 'a',
 6008: 'a',
 6642: 'a',
 6783: 'a',
 6858: 'a',
 6980: 'a',
 7125: 'a',
 7367: 'a',
 7422: 'a',
 7831: 'a',
 8124: 'a',
 8209: 'a',
 8320: 'a',
 8594: 'a',
 8700: 'a',
 8762: 'a',
 9184: 'a',
 9816: 'a',
 10048: 'a',
 10083: 'a',
 10111: 'a',
 10262: 'a',
 10358: 'a',
 10790: 'a',
 11049: 'a',
 11091: 'a',
 11507: 'a',
 11596: 'a',
 11660: 'a',
 12094: 'a',
 12260: 'a',
 12291: 'a',
 12491: 'a',
 12602: 'a',
 13193: 'a',
 13284: 'a',
 13321: 'a',
 13940: 'a',
 13959: 'a',
 13990: 'a',
 14131: 'a',
 14466: 'a',
 14938: 'a',
 15030: 'a',
 15046: 'a',
 15099: 'a',
 15166: 'a',
 15789: 'a',
 15854: 'a',
 16053: 'a',
 16725: 'a',
 16757: 'a',
 16797: 'a',
 17142: 

In [104]:
import pandas as pd
uid_list = [1824575646,289899198,1012024248]
for uid in uid_list:
    data_list = []
    topKRecs = model.recommendProducts(uid,30)
    for rec in topKRecs:
        data_list.append({'appid':rec.product,'usage_average_hours':rec.rating,'app_type':appid2class[rec.product] if rec.product in appid2class.keys() else 'unknown','data type':'recommend'})
    appsForUser = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(uid)
    print(f'源数据中用户(userId={uid},type={uid2label[uid]})使用的app(item)：')
    for i in sorted(appsForUser[0],key=lambda x : x.rating,reverse=True):
        data_list.append({'appid':i.product,'usage_average_hours':i.rating,'app_type':appid2class[i.product] if i.product in appid2class.keys() else 'unknown','data type':'meta'})
    tmp = pd.DataFrame(data_list)
    print(tmp)
    tmp.to_csv('/data/mobile/processed_data/user_%s_%s.csv'%(uid,uid2label[uid]))

源数据中用户(userId=1824575646,type=2)使用的app(item)：
    appid  usage_average_hours app_type  data type
0   16065             2.052123  unknown  recommend
1   10062             1.571900  unknown  recommend
2   19011             1.516709        t  recommend
3    3309             1.478687        f  recommend
4   27463             1.449437  unknown  recommend
..    ...                  ...      ...        ...
71   4664             0.000028  unknown       meta
72   7525             0.000028        s       meta
73  21526             0.000028  unknown       meta
74   1591             0.000019        c       meta
75  20124             0.000009  unknown       meta

[76 rows x 4 columns]
源数据中用户(userId=289899198,type=0)使用的app(item)：
    appid  usage_average_hours app_type  data type
0   17735             1.040475  unknown  recommend
1   53723             1.010017  unknown  recommend
2    1105             1.000623  unknown  recommend
3   27690             0.983041  unknown  recommend
4     936          